<a href="https://colab.research.google.com/github/marcusborela/Aprendizado-Profundo-Unicamp/blob/main/My_hw_openqa_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Few-shot OpenQA with ColBERT retrieval

In [1]:
__author__ = "Christopher Potts and Omar Khattab"
__version__ = "CS224u, Stanford, Spring 2022"

# fonte: https://github.com/cgpotts/cs224u/blob/master/hw_openqa.ipynb

## Contents

1. [Contents](#Contents)
1. [Visão geral](#Overview)
1. [Set-up](#Set-up)
    1. [Google Colab set-up](#Google-Colab-set-up)
    1. [General set-up](#General-set-up)
    1. [Language model set-up](#Language-model-set-up)
    1. [ColBERT set-up](#ColBERT-set-up)
1. [Language models](#Language-models)
    1. [Answerhood](#Answerhood)
    1. [Eleuther models from Hugging Face](#Eleuther-models-from-Hugging-Face)
    1. [GPT-3](#GPT-3)
1. [SQuAD](#SQuAD)
    1. [SQuAD dev](#SQuAD-dev)
    1. [SQuAD dev sample](#SQuAD-dev-sample)
    1. [SQuAD train](#SQuAD-train)
1. [Evaluation](#Evaluation)
1. [Open QA with no context](#Open-QA-with-no-context)
1. [Few-shot QA](#Few-shot-QA)
1. [ColBERT](#ColBERT)
    1. [ColBERT parameters](#ColBERT-parameters)
    1. [ColBERT index](#ColBERT-index)
    1. [Search](#Search)
    1. [Retrieval evaluation](#Retrieval-evaluation)
1. [Zero-shot OpenQA with ColBERT retrieval](#Zero-shot-OpenQA-with-ColBERT-retrieval)
1. [Homework questions](#Homework-questions)
    1. [Few-shot OpenQA with no context [2 points]](#Few-shot-OpenQA-with-no-context-[2-points])
    1. [Few-shot OpenQA [2 points]](#Few-shot-OpenQA-[2-points])
    1. [Answer scoring [2 points]](#Answer-scoring-[2-points])
    1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

The goal of this homework is to explore few-shot (or, prompt-based) learning in the context of open-domain question answering. This is an exciting area that brings together a number of recent task ideas and modeling innovations.

Our core task is __open-domain question answering (OpenQA)__. In this task, all that is given by the dataset is a question text, and the task is to answer that question. By contrast, in modern QA tasks, the dataset provides a text and a gold passage, with a guarantee that the answer will be a substring of the passage. 

OpenQA is substantially harder than standard QA. The usual strategy is to use a _retriever_ to find passages in a large collection of texts and train a _reader_ to find answers in those passages. This means we have no guarantee that the retrieved passage will contain the answer we need. If we don't retrieve a passage containing the answer, our reader has no hope of succeeding. Although this is challenging, it is much more realistic and widely applicable than standard QA. After all, with the right retriever, an OpenQA system could be deployed over the entire Web.

The task posed by this homework is harder even than OpenQA. We are calling this task __few-shot OpenQA__. The defining feature of this task is that the reader is simply a general purpose autoregressive language model. It accepts string inputs (prompts) and produces text in response. It is not trained to answer questions per se, and nothing about its structure ensures that it will respond with a substring of the prompt corresponding to anything like an answer.

__Few-shot QA__ (but not OpenQA!) is explored in the famous GPT-3 paper ([Brown et al. 2020](https://arxiv.org/abs/2005.14165)). The authors are able to get traction on the problem using GPT-3, an incredible finding. Our task here – __few-shot OpenQA__ – pushes this even further by retrieving passages to use in the prompt rather than assuming that the gold passage can be used in the prompt. If we can make this work, then it should be a major step towards flexibly and easily deploying QA technologies in new domains.

In summary:

| Task             | Passage given | Task-specific reader training |Task-specific retriever training  | 
|-----------------:|:-------------:|:-----------------------------:|:--------------------------------:|
| QA               | yes           | yes                           | n/a                              |
| OpenQA           | no            | yes                           | maybe                            |
| Few-shot QA      | yes           | no                            | n/a                              |
| Few-shot OpenQA  | no            | no                            | maybe                            | 

Just to repeat: your mission (should you choose to accept it!) is to explore the final line in this table. The core notebook and assignment don't address the issue of training the retriever in a task-specific way, but we've given some pointers on this in the context of [the original system question at the bottom of this notebook](#Your-original-system-[3-points]).

As usual, this notebook sets up the task and provides starter code. We proceed through a series of approaches:

* _Open QA with no context_: the prompt consists of the question, and we just see what comes back. This is not particularly fair to the system since it doesn't unambiguously convey what we want it to do, but it's a start.

* _Few-shot QA_: the prompt contains one or more examples formatted so as to indirectly convey what we want the system to do, and it uses the gold passage associated with the example. This is the approach of the GPT-3 paper. It works only for datasets with gold passages.

* _Open QA with ColBERT retrieval_: This is roughly as in the previous case, but now we presume no access to a gold passage for our example. Rather, we retrieve a passage from a large corpus using the neural information retrieval model ColBERT.

The above examples are followed by some assignment questions aimed at helping you to think creatively about the problem. These problems improve on the above approaches in various ways.

All of this culminates in an original system question and some code and unlabeled data (here, just a list of questions) for the bake-off.

It is a requirement of the bake-off that a pure autoregressive model be used. In particular, trained QA systems cannot be used at all. See the original system question at the bottom of this message for the full list of allowed models.

Note: the models we are working with here are _big_. This poses a challenge that is increasingly common in NLP: you have to pay one way or another. You can pay to use the GPT-3 API, or you can pay to use an Eleuther model on a heavy-duty cluster computer, or you can pay with time by using an Eleuther model on a more modest computer. If none of these options is palatable, you might consider instead doing the [color reference assignment](hw_colors.ipynb)!

## Set-up

### Google Colab set-up

We have sought to make this notebook self-contained so that it can easily be run as a Google Colab. If you are running it in Colab, make sure to select a GPU instance. The notebook will run on a CPU-only instance or CPU-only machine, but it should be much faster with GPU support.

The following are all installed as part of course set-up, but you'll want to run this cell if you are working in a Colab:

In [2]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
     |████████████████████████████████| 6.6 MB 81.2 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.2 MB/s 
     |████████████████████████████████| 140 kB 71.5 MB/s 
     |████████████████████████████████| 1.1 MB 79.6 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 212 kB 73.6 MB/s 
     |████████████████████████████████| 127 kB 53.5 MB/s 
     |████████████████████████████████| 271 kB 50.2 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 144 kB 80.1 MB/s 
     |████████████████████████████████| 112 kB 52.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
  Attempting unin

In [4]:
!pip install ujson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 1.9 MB/s 


In [5]:
!pip install gitpython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 181 kB 4.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


Se necessário, instalar:

* !pip install torch==1.10.0
* !pip install spacy
* 

If you are indeed on a GPU machine, then run the following to ensure complete CUDA support:

In [6]:
import torch

if torch.cuda.is_available():
    !pip install cupy-cuda111

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


If the above doesn't work, it might be because you don't have CUDA version 11.1. Run 

In [7]:
import torch

if torch.cuda.is_available():
    !nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


and then install the corresponding `cupy-cuda`. See [this table](https://docs.cupy.dev/en/stable/install.html#installing-cupy-from-pypi) for details on which one to install for different scenarios.

### General set-up

In [8]:
import collections
from contextlib import nullcontext
from collections import namedtuple
from datasets import load_dataset
import datasets
import json
import numpy as np
import random
import re 
import string
import torch
from typing import List
from getpass import getpass

Try to set all the seeds for reproducibility (won't extend to GPT-3):

In [9]:
seed = 1

np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

The following should install the version of [Faiss](https://github.com/facebookresearch/faiss) that will cooperate with your set-up:

In [10]:
import torch

if torch.cuda.is_available():
    !pip install faiss-gpu==1.7.0
else:
    !pip install faiss-cpu==1.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89.4 MB 168 kB/s 


### Language model set-up

To use the GPT-3 API, install the OpenAI library:

In [11]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 916 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 7.0 MB/s 
  Created wheel for openai: filename=openai-0.19.0-py3-none-any.whl size=53535 sha256=80b0d0eea1f99a34ee34e62aa3883a6dc4be052023908c06e72464260a829ffc
  Stored in directory: /root/.cache/pip/wheels/94/b5/c0/928013bd6418b23b9c5d89fb24cdeb1faae899c11377d69609
Successfully built openai


In [12]:
import openai
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [13]:
#datasets.utils.logging.set_verbosity_error()
#transformers.utils.logging.set_verbosity_error()
datasets.utils.logging.set_verbosity_warning()
transformers.utils.logging.set_verbosity_info()

### ColBERT set-up

Our retriever will be a ColbERT-based model ([Khattab and Zaharia 2020](https://arxiv.org/abs/2004.12832)). ColBERT is a powerful neural information retrieval (Neural IR) model that has proven extremely successful in retrieval applications and as a component in a variety of different systems for OpenQA and other knowledge-intensive tasks (e.g., [Khattab et al. 2021a](https://aclanthology.org/2021.tacl-1.55/); [Khattab et al. 2021b](https://proceedings.neurips.cc/paper/2021/hash/e8b1cbd05f6e6a358a81dee52493dd06-Abstract.html); [Santhanam, Khattab, et al. 2021](https://arxiv.org/abs/2112.01488)).

The following will clone the ColBERTv2 repository for use in this notebook:

In [14]:
!git clone -b cpu_inference https://github.com/stanford-futuredata/ColBERT.git

Cloning into 'ColBERT'...
remote: Enumerating objects: 772, done.
remote: Counting objects: 100% (395/395), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 772 (delta 279), reused 324 (delta 249), pack-reused 377
Receiving objects: 100% (772/772), 306.28 KiB | 2.55 MiB/s, done.
Resolving deltas: 100% (425/425), done.


In [15]:
import os
import sys
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [16]:
sys.path.insert(0, 'ColBERT/')
#print(sys.path)

In [17]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Collection
from colbert.searcher import Searcher
from utility.utils.dpr import has_answer, DPR_normalize

ColBERT/colbert/indexing/codecs/residual.py:10: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(not torch.cuda.is_available(), "cupy must be installed in GPU mode")


## Language models

In few-shot OpenQA, the language model (LM) must read in a prompt and answer the question posed somewhere in the prompt. We propose two basic strategies:

* [EleutherAI](https://www.eleuther.ai/) has released GPT-2-style models in a variety of sizes. These are free to use and easy to use via Hugging Face, and the larger ones very effective for our task, with GPT-J competitive with GPT-3 even though it has only 6B parameters (vs. 145B for GPT-3). The downside here is that the larger models in this family might be very slow and very difficult to work with unless you have access to really impressive GPU hardware. In testing with the free version of Google Colab, we were basically able to do everything we needed to do for the 1.3B parameter model, but the larger one caused too many problems to be viable.

* OpenAI has outstanding API access to GPT-3. You can sign up for [a free account](https://beta.openai.com/signup), and, as of this writing, you get US$18 in credit when you sign up. This is more than enough for the current assignment provided you are careful about how much testing you do. The benefits here are that the API is blazingly fast and requires nothing of your computer in terms of GPU support, and you're getting responses from a 145B parameter model that is truly exceptional.

Our suggestion is to do basic development with `"gpt-neo-125M"`, scale up to `"gpt-neo-1.3B"` once you have a sense for what your original system will be like, and then do your final bake-off entry with GPT-3. The functions `run_eleuther` and `run_gpt3` defined below are totally interchangeable, so this kind of development path should be easy to take.

### Answerhood

In [18]:
def _find_generated_answer(tokens, newline="\n" ): 
    """Our LMs tend to insert initial newline characters before
    they begin generating text. This function ensures that we 
    properly capture the true first line as the answer while
    also ensuring that token probabilities are aligned."""        
    answer_token_indices = []
    char_seen = False            
    for i, tok in enumerate(tokens):
        # This is the main condition: a newline that isn't an initial
        # string of newlines:
        if tok == newline and char_seen:
            break
        # Keep the initial newlines for consistency:
        elif tok == newline and not char_seen:
            answer_token_indices.append(i)
        # Proper tokens:
        elif tok != newline:
            char_seen = True
            answer_token_indices.append(i)
    return answer_token_indices 

### Eleuther models from Hugging Face

In [19]:
# "gpt-neo-125M" "gpt-neo-1.3B" "gpt-neo-2.7B" "gpt-j-6B"
eleuther_model_name = "gpt-neo-125M"

eleuther_tokenizer = AutoTokenizer.from_pretrained(
    f"EleutherAI/{eleuther_model_name}", 
    padding_side="left", 
    padding='longest', 
    truncation='longest_first', max_length=2000)
eleuther_tokenizer.pad_token = eleuther_tokenizer.eos_token

eleuther_model = AutoModelForCausalLM.from_pretrained(
    f"EleutherAI/{eleuther_model_name}")

device = "cuda" if torch.cuda.is_available() else "cpu"
eleuther_model = eleuther_model.to(device)

https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpzcb7mssp


Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/3cc88b3aa29bb2546db2dc21783292e2a086bb7158c7b5ceddeb24158a85c183.e74f7c3643ee79eb023ead36008be72fe726dada60fa3b2a0569925cfefa1e74
creating metadata file for /root/.cache/huggingface/transformers/3cc88b3aa29bb2546db2dc21783292e2a086bb7158c7b5ceddeb24158a85c183.e74f7c3643ee79eb023ead36008be72fe726dada60fa3b2a0569925cfefa1e74
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7sbidvcr


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
creating metadata file for /root/.cache/huggingface/transformers/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
loading configuration file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
Model config GPTNeoConfig {
  "_name_or_path": "EleutherAI/gpt-neo-125M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",

...

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpispgdkns


Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
creating metadata file for /root/.cache/huggingface/transformers/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/08c00c4159e921d4c941ac75732643373aba509d9b352a82bbbb043a94058d98.a552555fdda56a1c7c9a285bccfd44ac8e4b9e26c8c9b307831b3ea3ac782b45
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
l

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/b0ace3b93ace62067a246888f1e54e2d3ec20807d4d3e27ac602eef3b7091c0b.6525df88f1d5a2d33d95ce2458ef6af9658fe7d1393d6707e0e318779ccc68ff
creating metadata file for /root/.cache/huggingface/transformers/b0ace3b93ace62067a246888f1e54e2d3ec20807d4d3e27ac602eef3b7091c0b.6525df88f1d5a2d33d95ce2458ef6af9658fe7d1393d6707e0e318779ccc68ff
loading weights file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/b0ace3b93ace62067a246888f1e54e2d3ec20807d4d3e27ac602eef3b7091c0b.6525df88f1d5a2d33d95ce2458ef6af9658fe7d1393d6707e0e318779ccc68ff
All model checkpoint weights were used when initializing GPTNeoForCausalLM.

All the weights of GPTNeoForCausalLM were initialized from the model checkpoint at EleutherAI/gpt-neo-125M.
If your task is similar to the task the model of the checkpoint was train

In [20]:
def run_eleuther(prompts, temperature=0.1, top_p=0.95, **generate_kwargs): 
    """
    Parameters
    ----------
    prompts : iterable of str
    temperature : float
        It seems best to set it low for this task!
    top_p : float
       
    For options for `generate_kwargs`, see:
    
    https://huggingface.co/docs/transformers/master/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate
    
    Options that are likely to be especially relevant include 
    `temperature`, `length_penalty`, and the parameters that
    determine the decoding strategy. With `num_return_sequences > 1`,
    the default parameters in this function do multinomial sampling.
    
    Returns
    -------
    list of dicts
    
    {"prompt": str, 
     "generated_text": str, "generated_tokens": list of str, "generated_probs": list of float,
     "answer": str, "answer_tokens": list of str, "answer_probs": list of float
    }
         
    """
    prompt_ids = eleuther_tokenizer(
        prompts, return_tensors="pt", padding=True).input_ids.to(device)
        
    with torch.inference_mode():
        # Automatic mixed precision if possible.
        with torch.cuda.amp.autocast() if torch.cuda.is_available() else nullcontext():
            model_output = eleuther_model.generate(
                prompt_ids,
                temperature=temperature,
                do_sample=True,
                top_p=top_p,           
                max_new_tokens=16,
                num_return_sequences=1,                
                pad_token_id=eleuther_tokenizer.eos_token_id, 
                return_dict_in_generate=True,
                output_scores=True,
                **generate_kwargs)
        
    # Converting output scores using the helpful recipe here:
    # https://discuss.huggingface.co/t/generation-probabilities-how-to-compute-probabilities-of-output-scores-for-gpt2/3175
    gen_ids = model_output.sequences[:, prompt_ids.shape[-1] :]
    gen_probs = torch.stack(model_output.scores, dim=1).softmax(-1)
    gen_probs = torch.gather(gen_probs, 2, gen_ids[:, :, None]).squeeze(-1)
    
    # Generated texts, including the prompts:
    gen_texts = eleuther_tokenizer.batch_decode(
        model_output.sequences, skip_special_tokens=True)
    
    data = []     
    iterator = zip(prompts, gen_ids, gen_texts, gen_probs)    
    for prompt, gen_id, gen_text, gen_prob in iterator:       
        gen_tokens = eleuther_tokenizer.convert_ids_to_tokens(gen_id)
        generated_text = gen_text[len(prompt): ]
        gen_prob = [float(x) for x in gen_prob.cpu().numpy()] # float for JSON storage
        ans_indices = _find_generated_answer(gen_tokens, newline="Ċ")
        answer_tokens = [gen_tokens[i] for i in ans_indices]
        answer_probs = [gen_prob[i] for i in ans_indices]
        answer = "".join(answer_tokens).replace("Ġ", " ").replace("Ċ", "\n")                                       
        data.append({
            "prompt": prompt,
            "generated_text": generated_text,
            "generated_tokens": gen_tokens,
            "generated_probs": gen_prob,
            "generated_answer": answer,
            "generated_answer_probs": answer_probs,
            "generated_answer_tokens": answer_tokens})                        

    return data

In [21]:
eleuther_ex = run_eleuther([    
    "What year was Stanford University founded?", 
    "In which year did Stanford first enroll students?"])

eleuther_ex

[{'generated_answer': '\n\nThe Stanford University Foundation is a non-profit organization that provides scholarships to',
  'generated_answer_probs': [1.0,
   1.0,
   0.9111328125,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.85986328125,
   1.0,
   1.0,
   1.0,
   1.0,
   0.90966796875,
   0.7216796875,
   0.671875],
  'generated_answer_tokens': ['Ċ',
   'Ċ',
   'The',
   'ĠStanford',
   'ĠUniversity',
   'ĠFoundation',
   'Ġis',
   'Ġa',
   'Ġnon',
   '-',
   'profit',
   'Ġorganization',
   'Ġthat',
   'Ġprovides',
   'Ġscholarships',
   'Ġto'],
  'generated_probs': [1.0,
   1.0,
   0.9111328125,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.85986328125,
   1.0,
   1.0,
   1.0,
   1.0,
   0.90966796875,
   0.7216796875,
   0.671875],
  'generated_text': '\n\nThe Stanford University Foundation is a non-profit organization that provides scholarships to',
  'generated_tokens': ['Ċ',
   'Ċ',
   'The',
   'ĠStanford',
   'ĠUniversity',
   'ĠFoundation',
   'Ġis',
   'Ġa',
   'Ġnon',
   '

### GPT-3

In [22]:
# Fill this in with the value from your OpenAI account. First
# verify that your account is set up with a spending limit that
# you are comfortable with. If you just opened your account,
# you should have $18 in credit and so won't need to supply any
# payment information.
openai.api_key = getpass('Informe openai.api_key ')

Informe openai.api_key ··········


In [23]:
def run_gpt3(prompts, engine="text-curie-001", temperature=0.1, top_p=0.95, **gpt3_kwargs):
    """To use this function, sign up for an OpenAI account at
        
    https://beta.openai.com/signup
    
    That should give you $18 in free credits, which is more than enough
    for this assignment assuming you are careful with testing.
    
    Once your account is set up, you can get your API key from your 
    account dashboard and paste it in below as the value of 
    `openai.api_key`.
    
    Parameters
    ----------
    prompts : iterable of str
    engine : str
        This has to be one of the models whose name begins with "text".
        The "instruct" class of models can't be used, since they seem
        to depend on some kinds of QA-relevant supervision.        
        For options, costs, and other details: 
        https://beta.openai.com/docs/engines/gpt-3                
    temperature : float
        It seems best to set it low for this task!
    top_p : float
        
    For information about values for `gpt3_kwargs`, see
    
    https://beta.openai.com/docs/api-reference/completions
    
    Returns
    -------
    list of dicts   
    
    """

    assert engine.startswith("text"), \
        "Please use an engine whose name begins with 'text'."
        
    response = openai.Completion.create(
        engine=engine,       
        prompt=prompts,
        temperature=temperature,
        top_p=top_p,
        echo=False,   # This function will not work
        logprobs=1,   # properly if any of these
        n=1,          # are changed!
        **gpt3_kwargs)
    
    # From here, we parse each example to get the values
    # we need:
    data = []
    for ex, prompt in zip(response["choices"], prompts):
        tokens = ex["logprobs"]["tokens"]
        logprobs = ex["logprobs"]["token_logprobs"]        
        probs = list(np.exp(logprobs))
        if "<|endoftext|>" in tokens:
            end_i = tokens.index("<|endoftext|>")
            tokens = tokens[ : end_i]  # This leaves off the "<|endoftext|>"
            probs = probs[ : end_i]    # token -- perhaps dubious.
        ans_indices = _find_generated_answer(tokens)
        answer_tokens = [tokens[i] for i in ans_indices]
        answer_probs = [probs[i] for i in ans_indices]
        answer = "".join(answer_tokens)        
        data.append({
            "prompt": prompt,
            "generated_text": ex["text"],
            "generated_tokens": tokens,
            "generated_probs": probs,
            "generated_answer": answer,
            "generated_answer_tokens": answer_tokens,
            "generated_answer_probs": answer_probs})
        
    return data             

In [24]:
gpt3_ex = run_gpt3([
     "What year was Stanford University founded?",
     "In which year did Stanford first enroll students?"])

gpt3_ex

[{'generated_answer': '\n\n1891',
  'generated_answer_probs': [0.9991431717871095,
   0.9997846349243882,
   0.5741379962336021,
   0.39573849184479504],
  'generated_answer_tokens': ['\n', '\n', '18', '91'],
  'generated_probs': [0.9991431717871095,
   0.9997846349243882,
   0.5741379962336021,
   0.39573849184479504],
  'generated_text': '\n\n1891',
  'generated_tokens': ['\n', '\n', '18', '91'],
  'prompt': 'What year was Stanford University founded?'},
 {'generated_answer': '\n\nStanford first enrolled students in 1891.',
  'generated_answer_probs': [0.9993135207847473,
   0.9998701106163566,
   0.4383535510647387,
   0.9999952788481447,
   0.8323608735103043,
   0.9814090478885719,
   0.9988871687564463,
   0.9936751893545676,
   0.9917295955365046,
   0.6287194545379907,
   0.9849397876183434],
  'generated_answer_tokens': ['\n',
   '\n',
   'Stan',
   'ford',
   ' first',
   ' enrolled',
   ' students',
   ' in',
   ' 18',
   '91',
   '.'],
  'generated_probs': [0.99931352078474

## SQuAD

Our core development dataset is [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/). We chose this dataset because it is well-known and widely used, and it is large enough to support lots of meaningful development work, without, though, being so large as to require lots of compute power. It is also useful that it has gold passages supporting the standard QA formulation, so we can see how well our LM performs with an "oracle" retriever that always retrieves the gold passage.

In [25]:
squad = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

The following utility just reads a SQuAD split in as a list of `SquadExample` instances:

In [ ]:
SquadExample = namedtuple("SquadExample",  "id title context question answers")

In [ ]:
def get_squad_split(squad, split="validation"):
    """
    Use `split='train'` for the train split.
    
    Returns
    -------
    list of SquadExample named tuples with attributes
    id, title, context, question, answers
    
    """    
    fields = squad[split].features
    data = zip(*[squad[split][field] for field in fields])
    return [SquadExample(eid, title, context, question, answers["text"]) 
            for eid, title, context, question, answers in data]

### SQuAD dev

In [ ]:
squad_dev = get_squad_split(squad)

In [29]:
squad_dev[0]

SquadExample(id='56be4db0acb8001400a502ec', title='Super_Bowl_50', context='Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.', question='Which NFL team represented the AFC at Super Bowl 50?', answers=['Denver Broncos', 'Denver Broncos', 'Denver Broncos'])

### SQuAD dev sample

We'll use this fixed but presumably quite random set of examples for exploration and system development:

In [30]:
dev_exs = sorted(squad_dev, key=lambda x: hash(x.id))[: 200]

### SQuAD train

To build few-shot prompts, we will often sample SQuAD train examples, so we load that split here:

In [ ]:
squad_train = get_squad_split(squad, "train")

## Evaluation

Our evaluation protocols are the standard ones for SQuAD and related tasks: exact match of the answer (EM) and token-level F1.

We say further that the predicted answer is the first line of generated text after the prompt.

The following evaluation code is taken from the [apple/ml-qrecc](https://github.com/apple/ml-qrecc/blob/main/utils/evaluate_qa.py) repository. It performs very basic string normalization before doing the core comparisons.

In [ ]:
def normalize_answer(s: str) -> str:
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s: str) -> List[str]:
    """Normalize string and split string into tokens."""
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold: str, a_pred: str) -> int:
    """Compute the Exact Match score."""
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1_from_tokens(gold_toks: List[str], pred_toks: List[str]) -> float:
    """Compute the F1 score from tokenized gold answer and prediction."""
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())

    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def compute_f1(a_gold: str, a_pred: str) -> float:
    """Compute the F1 score."""
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    return compute_f1_from_tokens(gold_toks, pred_toks)

The following is our general evaluation function. We will make extensive use of it to evaluate different systems:

In [ ]:
def evaluate(examples, prompts, gens):
    """Generic evalution function.
    
    Parameters
    ----------
    examples: iterable of `SquadExample` instances
    prompts: list of str
    preds: list of LM-generated texts to evaluate as answers
    
    Returns
    -------
    dict with keys "em_per", "macro_f1", "examples", where
    each "examples" value is a dict
    
    """        
    results = []
    for ex, prompt, gen in zip(examples, prompts, gens):
        answers = ex.answers
        pred = gen['generated_answer']
        # The result is the highest EM from the available answer strings:
        em = max([compute_exact(ans, pred) for ans in answers])
        f1 = max([compute_f1(ans, pred) for ans in answers])
        gen.update({
            "id": ex.id, 
            "question": ex.question, 
            "prediction": pred, 
            "answers": answers, 
            "em": em,
            "f1": f1
        })
        results.append(gen)
    data = {}        
    data["macro_f1"] = np.mean([d['f1'] for d in results])
    data["em_per"] = sum([d['em'] for d in results]) / len(results)
    data["examples"] = results
    return data

Here is a highly simplified example to help make the logic behind `evaluate` clearer:    

In [ ]:
ex = namedtuple("SquadExample",  "id title context question answers")

examples = [
    ex("0", "CS224u", 
       "The course to take is NLU!", 
       "What is the course to take?", 
       ["NLU", "CS224u"])]

prompts = ["Dear model, Please answer this question!\n\nQ: What is the course to take?\n\nA:"]

gens = [{"generated_answer": "NLU", "generated_text": "NLU\nWho am I?"}]

evaluate(examples, prompts, gens)

The bake-off uses `macro_f1` as the primary metric.

## Open QA with no context

We now have all the pieces we need to begin building few-shot OpenQA systems. Our first system is the simplest and most naive: we simply feed the question text in as the prompt and hope that the model provides an answer as the first line of its generated text.

In [ ]:
def evaluate_no_context(examples, gen_func=run_eleuther, batch_size=20):
    prompts = [] 
    gens = []
    for i in range(0, len(examples), batch_size):
        ps = [ex.question for ex in examples[i: i+batch_size]]
        gs = gen_func(ps)        
        prompts += ps
        gens += gs    
    return evaluate(examples, prompts, gens)    

In [ ]:
%%time
nocontext_results = evaluate_no_context(dev_exs)

print(nocontext_results['macro_f1'])

In [ ]:
%%time
nocontext_results_gpt3 = evaluate_no_context(dev_exs, gen_func=run_gpt3)

print(nocontext_results_gpt3['macro_f1'])

## Few-shot QA

The above formulation is not especially fair to our model, since it doesn't convey anything about the intended structure of the prompt. We want the model to give us an answer to the input question, but we didn't specify that goal unambiguously. Perhaps we were looking for commentary on the question, or a count of the number of tokens it contains, or a passage containing the question string, or something else entirely.

In few-shot QA, we construct a prompt that is intended to convey our intentions more clearly. The first part of the prompt gives some examples of what we want, and the final part provides the set-up for our actual question. In the current formulation, we assume access to the gold passage. For example, if our example of interest is

```
Title: CS224u

Background: The course to take is NLU!

Q: What is the course to take?
```

with gold answer ```NLU```, then we would create a prompt with, say, 2 additional examples preceding this, to yield a full prompt like this:

```
Title: Pragmatics

Background: Pragmatics is the study of language use.

Q: What is pragmatics?

A: The study of language use

Title: Bert

Background: Bert is a Muppet who is lives with Ernie.

Q: Who is Bert?

A: Bert is a  Muppet

Title: CS224u

Background: The course to take is NLU!

Q: What is the course to take?

A:
```
This is essentially the formulation used in the GPT-3 paper for SQuAD. The context examples are drawn randomly from the SQuAD train set. We will adopt this same protocol for now. (You might revisit this in the context of your original system.)

In [ ]:
def build_few_shot_qa_prompt(ex, squad_train, n_context=2, joiner="\n\n"):
    segs = []
    train_exs = random.sample(squad_train, k=n_context)    
    for t in train_exs:
        segs += [
            f"Title: {t.title}",
            f"Background: {t.context}",
            f"Q: {t.question}",
            f"A: {t.answers[0]}"
        ]
    segs += [
        f"Title: {ex.title}",
        f"Background: {ex.context}",
        f"Q: {ex.question}",
        f"A:"
    ]
    return joiner.join(segs)                

Here's the sort of output we get with `n_context=1`:

In [ ]:
print(build_few_shot_qa_prompt(dev_exs[0], squad_train, n_context=1))

In [ ]:
def evaluate_few_shot_qa(examples, squad_train, gen_func=run_eleuther, batch_size=20, n_context=2):
    prompts = []
    gens = []
    for i in range(0, len(examples), batch_size):
        batch = examples[i: i+batch_size]
        ps = [build_few_shot_qa_prompt(ex, squad_train, n_context=n_context) for ex in batch]        
        gs = gen_func(ps)       
        prompts += ps
        gens += gs
    return evaluate(examples, prompts, gens)

In [ ]:
%%time
few_shot_qa_results = evaluate_few_shot_qa(dev_exs, squad_train, n_context=1)

print(few_shot_qa_results['macro_f1'])

In [ ]:
%%time
few_shot_qa_results_gpt3 = evaluate_few_shot_qa(
    dev_exs, squad_train, n_context=1, gen_func=run_gpt3)

print(few_shot_qa_results_gpt3['macro_f1'])

## ColBERT

It's now just a short step to our core task, few-shot OpenQA. We just need to give up our beloved gold passage and instead try to retrieve the right passage or passages from a corpus. 

The first step is instantiating the ColBERT retriever and loading in an index. Our ColBERT retriever was initially trained on MS MARCO, and we have pre-indexed a collection of 100K documents that we know to be well-aligned with SQuAD and with the dataset used for the bake-off assessment. (See [the original system question](#Your-original-system-[3-points]) for tips on creating your own index.)

In [ ]:
index_home = os.path.join("experiments", "notebook", "indexes")

### ColBERT parameters

In [ ]:
if not os.path.exists(os.path.join("data", "openqa", "colbertv2.0.tar.gz")):
    !mkdir -p data/openqa
    # ColBERTv2 checkpoint trained on MS MARCO Passage Ranking (388MB compressed)
    !wget https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz -P data/openqa/
    !tar -xvzf data/openqa/colbertv2.0.tar.gz -C data/openqa/

If something went wrong with the above, you can just download the file https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz, unarchive it, and move the resulting `colbertv2.0` directory into the `data/openqa` directory.

### ColBERT index

In [ ]:
if not os.path.exists(os.path.join(index_home, "cs224u.collection.2bits.tgz")):
    !wget https://web.stanford.edu/class/cs224u/data/cs224u.collection.2bits.tgz -P experiments/notebook/indexes
    !tar -xvzf experiments/notebook/indexes/cs224u.collection.2bits.tgz -C experiments/notebook/indexes

If something went wrong with the above, download the file https://web.stanford.edu/class/cs224u/data/cs224u.collection.2bits.tgz, unarchive it, and move the resulting `cs224u.collection.2bits` directory into the `experiments/notebook/indexes` directory (which you will probably need to create).

In [ ]:
collection = os.path.join(index_home, "cs224u.collection.2bits", "cs224u.collection.tsv")

collection = Collection(path=collection)

f'Loaded {len(collection):,} passages'

In [ ]:
index_name = "cs224u.collection.2bits"

Now we create our `searcher`:

In [ ]:
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name)

### Search

Now that the index is loaded, you can do searches over it. The index is limited, but retrieval is very solid!

In [ ]:
query = "linguistics"

print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3) 

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t[{passage_rank}]\t{passage_score:.1f}\t {searcher.collection[passage_id]}")

### Retrieval evaluation

For more rigorous evaluations of the retriever alone, we can use Sucess@`k` defined relative to the SQuAD passages and answers. We say that we have a "success" if a passage in the top `k` retrieved passages contains any of the answers substrings, and Sucess@`k` is the percentage of such success cases. This is very heuristic (perhaps the answer string happens to occur somewhere in a completely irrelevant passage), but it can still be good guidance.

In [ ]:
def success_at_k(examples, k=20):
    scores = []
    for ex in examples: 
        scores.append(evaluate_retrieval_example(ex, k=5))
    return sum(scores) / len(scores)
        
    
def evaluate_retrieval_example(ex, k=20):    
    results = searcher.search(ex.question, k=k)
    for passage_id, passage_rank, passage_score in zip(*results):
        passage = searcher.collection[passage_id]
        score = has_answer([DPR_normalize(ans) for ans in ex.answers], passage)
        if score:
            return 1
    return 0

Here is Sucess@20 for the SQuAD dev set:

In [ ]:
%%time
if torch.cuda.is_available():
    # This will take a few hours on a CPU:
    print(success_at_k(squad_dev))
else:
    # This should be reasonably fast and yields the
    # same kind of result:
    print(success_at_k(dev_exs))

## Zero-shot OpenQA with ColBERT retrieval

We're now in a position to define a system that does our full few-shot OpenQA task. To get this started, we define just a version that doesn't include any SQuaD-training examples in the prompt. So this is really zero-shot OpenQA. (The homework asks you to move to the true few-shot setting.)

In [ ]:
def build_zero_shot_openqa_prompt(question, passage, joiner="\n\n"):
    title, context = passage.split(" | ", 1)
    segs = [
        f"Title: {title}",
        f"Background: {context}",
        f"Q: {question}",
        "A:"
    ]
    return joiner.join(segs)    

In [ ]:
def evaluate_zero_shot_openqa(examples, joiner="\n\n", gen_func=run_eleuther, batch_size=20):
    prompts = []
    gens = []
    for i in range(0, len(examples), batch_size):
        exs = examples[i: i+batch_size]
        results = [searcher.search(ex.question, k=1) for ex in exs]
        passages = [searcher.collection[r[0][0]] for r in results]
        ps = [build_zero_shot_openqa_prompt(ex.question, psg, joiner=joiner) 
              for ex, psg in zip(exs, passages)]
        gs = gen_func(ps)       
        prompts += ps
        gens += gs
    return evaluate(examples, prompts, gens)

In [ ]:
%%time
zero_shot_openqa_results = evaluate_zero_shot_openqa(dev_exs)

print(zero_shot_openqa_results['macro_f1'])

In [ ]:
%%time
zero_shot_openqa_results_gpt3 = evaluate_zero_shot_openqa(dev_exs, gen_func=run_gpt3)

zero_shot_openqa_results_gpt3['macro_f1']

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Few-shot OpenQA with no context [2 points]

In the section [Open QA with no context](#Open-QA-with-no-context) above, we simply prompted our LM with a question string and looked at what came back. This is arguably unfair to the LM, since we didn't convey anything about our intentions.

For a fairer assessment of what the LM alone can do, we should move to the few-shot setting by giving the model a few examples of what we have in mind. The idea here is to create prompts that look like this:

   ```   
   Q: What is pragmatics?

   A: The study of language use

   Q: Who is Bert?

   A: Bert is one of the Muppets.

   Q: What was Stanford University founded?
   
   A: 
   ```
   
This question asks you to write a function for creating such prompts, using SQuAD training examples, and a second function for evaluating this approach. The goal is to have a no context baseline for the other few-shot approaches we are considering.

__Task 1___: Complete the function `build_few_shot_no_context_prompt` so that it builds prompts like the above. You can use `test_build_few_shot_no_context_prompt` to check that your function is returning prompts in the desired format.

__Task 2__: Complete the function `evaluate_few_shot_no_context` so that you can evaluate this approach. You can use `test_evaluator` to check that your function is performing the desired kind of evaluation.

In [ ]:
def build_few_shot_no_context_prompt(question, train_exs, joiner="\n\n"):
    """No context few-shot OpenQA prompts.

    Parameters
    ----------
    question : str   
    train_exs : iterable of SQuAD train examples. These can be 
        obtained via a random sample 
        from `squad_train` as defined above.
    joiner : str
        The character to use to join pieces of the prompt into 
        a single str.

    Returns
    -------
    str, the prompt

    """
    ##### YOUR CODE HERE




In [ ]:
def test_build_few_shot_no_context_prompt(func):
    train_exs = [
        SquadExample(0, "T1", "Q1", "C1", ["A1"]),
        SquadExample(1, "T2", "Q2", "C2", ["A2"]),
        SquadExample(2, "T3", "Q3", "C3", ["A3"])]
    question = "My Q"
    result = func(question, train_exs, joiner="\n")
    expected = ""
    tests = [
        (1, "\n", 'Q: C1\nA: A1\nQ: My Q\nA:'),                
        (1, "\n\n", 'Q: C1\n\nA: A1\n\nQ: My Q\n\nA:'),
        (2, "\n", 'Q: C1\nA: A1\nQ: C2\nA: A2\nQ: My Q\nA:')]
    err_count = 0       
    for n_context, joiner, expected in tests:
        result = func(question, train_exs[: n_context], joiner=joiner)
        if result != expected:
            err_count +=1 
            print(f"Error:\n\nExpected:\n\n{expected}\n\nGot:\n\n{result}")    
    if err_count == 0:
        print("No errors detected in `build_few_shot_no_context_prompt`")     

In [ ]:
test_build_few_shot_no_context_prompt(build_few_shot_no_context_prompt)

In [ ]:
def evaluate_few_shot_no_context(
        examples,
        squad_train,
        batch_size=20,
        n_context=2,
        joiner="\n\n",
        gen_func=run_eleuther):
    """Evaluate a few-shot OpenQA with no context approach 
    defined by `build_few_shot_no_context_prompt` and `gen_func`.

    Parameters
    ----------
    examples : iterable of SQuAD train examples
        Presumably a subset of `squad_dev` as defined above.
    squad_train : iterable of SQuAD train examples
    batch_size : int
        Number of examples to send to `gen_func` at once.
    n_context : n
        Number of examples to use from `squad_train`.
    joiner : str
        Used by `build_few_shot_open_qa_prompt` to join segments
        of the prompt into a single str.
    gen_func : either `run_eleuther` or `run_gpt3`

    Returns
    -------
    dict as determined by `evaluate` above.

    """
    # A list of strings that you build and feed into `gen_func`.
    prompts = []

    # A list of dicts that you get from `gen_func`.
    gens = []

    # Iterate through the examples in batches:
    for i in range(0, len(examples), batch_size):
        # Sample some SQuAD training examples to use with
        # `build_few_shot_no_context_prompt` and `ex.question`,
        # run the resulting prompt through `gen_func`, and
        # add your prompts and results to `prompts` and `gens`.

        ##### YOUR CODE HERE



    # Return value from a call to `evalaute`, with `examples`
    # as provided by the user and the `prompts` and `gens`
    # you built:
    return evaluate(examples, prompts, gens)

In [ ]:
def test_evaluator(func):
    examples = [SquadExample(0, "T1", "Q1", "C1", ["A1"])]    
    squad_train = [SquadExample(0, "sT1", "sQ1", "sC1", ["sA1"])] 
    
    def gen_func(*prompts):
        return [{
            "generated_answer": "Constant output", 
            "generated_answer_tokens": ["Constant", "output"], 
            "generated_answer_probs": [0.1, 0.2]}]
    
    batch_size = 1    
    n_context = 1    
    joiner = "\n"
    result = func(
        examples, 
        squad_train, 
        batch_size=1, 
        n_context=1, 
        joiner=joiner, 
        gen_func=gen_func)
    expected_keys = {'em_per', 'examples', 'macro_f1'}
    result_keys = set(result.keys())     
    if expected_keys != result_keys:
        print(f"Unexpected keys in result. "
              f"Expected: {expected_keys}; Got: {result_keys}")
        return
    expected_ex_keys = {
        'f1', 'id', 'em', 'generated_answer_tokens', 'generated_answer_probs',
        'prediction', 'generated_answer', 'question', 'answers'}
    result_ex_keys = set(result["examples"][0].keys())
    if expected_ex_keys != result_ex_keys:
        print(f"Unexpected keys in result['examples']. "
              f"Expected: {expected_ex_keys}; Got: {result_ex_keys}")
        return
    print("No errors detected in `evaluate_few_shot_open_qa`")  

In [ ]:
test_evaluator(evaluate_few_shot_no_context)

### Few-shot OpenQA [2 points]

In the section [Few-shot QA](Few-shot-QA) above, we used SQuAD training examples to build prompts that we hope will help the model infer our intended semantics for the prompts themselves. When we moved to the open formulation of the problem, in [Open QA with ColBERT retrieval](Open-QA-with-ColBERT-retrieval), we forced the model to deal with prompts that lack these context clues. This is a "zero-shot" formulation of the problem. The goal of this homework problem is to improve that system so that it truly supports few-shot OpenQA.

__Task 1__: Complete the function `build_few_shot_open_qa_prompt` so that it builds prompts from a question, a passage, and a sample of SQuAD training examples. You can use `test_build_few_shot_open_qa_prompt` to check that your function is returning prompts in the desired format.

__Task 2__: Complete the function `evaluate_few_shot_open_qa` so that you can evaluate this approach. You can use `test_evaluator` from above to check that your function is performing the desired kind of evaluation.

We will be checking only that the tests pass. We will not be evaluating the quality of the results you obtain using this code.

In [ ]:
def build_few_shot_open_qa_prompt(question, passage, train_exs, joiner="\n\n"):
    """Few-shot OpenQA prompts.

    Parameters
    ----------
    question : str
    passage : str
        Presumably something retrieved via search.
    train_exs : iterable of SQuAD train examples
        These can be obtained via a random sample from 
        `squad_train` as defined above.
    joiner : str
        The character to use to join pieces of the prompt 
        into a single str.

    Returns
    -------
    str, the prompt

    """
    ##### YOUR CODE HERE




In [ ]:
def test_build_few_shot_open_qa_prompt(func):
    train_exs = [
        SquadExample(0, "T1", "Q1", "C1", ["A1"]),
        SquadExample(1, "T2", "Q2", "C2", ["A2"]),
        SquadExample(2, "T3", "Q3", "C3", ["A3"])]            
    question = "My Q"    
    passage = "Title | target passage"    
    tests = [
        (1, "\n", ('Title: T1\nBackground: Q1\nQ: C1\nA: A1\n'
                   'Title: Title\nBackground: target passage\nQ: My Q\nA:')),
        (1, "\n\n", ('Title: T1\n\nBackground: Q1\n\nQ: C1\n\nA: A1\n\n'
                     'Title: Title\n\nBackground: target passage\n\nQ: My Q\n\nA:')),
        (2, "\n", ('Title: T1\nBackground: Q1\nQ: C1\nA: A1\nTitle: T2\n'
                   'Background: Q2\nQ: C2\nA: A2\nTitle: Title\n'
                   'Background: target passage\nQ: My Q\nA:'))]
    err_count = 0       
    for n_context, joiner, expected in tests:
        result = func(question, passage, train_exs[: n_context], joiner=joiner)
        if result != expected:
            err_count +=1 
            print(f"Error:\n\nExpected:\n\n{expected}\n\nGot:\n\n{result}")    
    if err_count == 0:
        print("No errors detected in `build_few_shot_open_qa_prompt`")

In [ ]:
test_build_few_shot_open_qa_prompt(build_few_shot_open_qa_prompt)

In [ ]:
def evaluate_few_shot_open_qa(
        examples,
        squad_train,
        batch_size=20,
        n_context=2,
        joiner="\n\n",
        gen_func=run_eleuther):
    """Evaluate a few-shot OpenQA approach defined by 
    `build_few_shot_open_qa_prompt` and `gen_func`.

    Parameters
    ----------
    examples : iterable of SQuAD train examples
        Presumably a subset of `squad_dev` as defined above.
    squad_train : iterable of SQuAD train examples
    batch_size : int
        Number of examples to send to `gen_func` at once.
    joiner : str
        Used by `build_few_shot_open_qa_prompt` to join segments
        of the prompt into a single str.
    gen_func : either `run_eleuther` or `run_gpt3`

    Returns
    -------
    dict as determined by `evaluate` above.

    """
    # A list of strings that you build and feed into `gen_func`.
    prompts = []

    # A list of dicts that you get from `gen_func`.
    gens = []

    # Iterate through the examples in batches:
    for i in range(0, len(examples), batch_size):
        # Use the `searcher` defined above to get passages
        # using `ex.question` as the query, and use your
        # `build_few_shot_open_qa_prompt` to build prompts.

        ##### YOUR CODE HERE



    # Return value from a call to `evalaute`, with `examples`
    # as provided by the user and the `prompts` and `gens`
    # you built:
    return evaluate(examples, prompts, gens)

In [ ]:
test_evaluator(evaluate_few_shot_open_qa)

### Answer scoring [2 points]

We have so far been assuming that the top-ranked passage retrieved by ColBERT should be used in the prompt and that the single answer returned by the LM is our prediction. It may be possible to improve on this by scoring answers using the ColBERT scores and the probabilities returned by the LM. This question asks you to explore a basic approach to such scoring. The core scoring function:

$$
\textbf{score}_{\text{prompt-func}}(\textrm{answer}, \textrm{passage}, \textrm{question}) = 
P(\textrm{passage} \mid \textrm{question}) \cdot 
P(\textrm{answer} \mid \text{prompt-func}(\textrm{question}, \textrm{passage}) ) 
$$

where we estimate the two conditional probabilities as follows:

* $P(\textrm{passage} \mid \textrm{question})$ is defined only for the top $k$ passages and defined by the softmax of the top $k$ scores returned by the retriever.

* $P(\textrm{answer} \mid \text{prompt-func}(\textrm{question}, \textrm{passage}))$ is simply the product of the per-token probabilities of the generated answer given the prompt determined by $\text{prompt-func}(\textrm{question}, \textrm{passage})$. These values can be extracted from the return values of both `run_eleuther` and `run_gpt3` using the key `"generated_answer_probs"`. (Your prompt function might of course have other arguments not represented here.)

__Your task__: Implement this scoring function for an individual example. The two required pieces are `get_passages_with_scores` and `answer_scoring`. Starter code for each is below, and each has a unit test you can run to check your work.

(With this implemented, it is easy to create a new prediction function that uses the $\textrm{answer}$ from the highest-scoring $\textrm{answer}/\textrm{passage}$ pair as the prediction for input $\textrm{question}$. You are not required to implement such a prediction function, but you might do this as part of [your original system](#Your-original-system-[3-points]).)

In [ ]:
def get_passages_with_scores(question, k=5):
    """Pseudo-probabilities from the retriever.

    Parameters
    ----------
    question : str
    k : int
        Number of passages to retrieve.

    Returns
    -------
    passages (list of str), passage_probs (np.array)

    """
    # Use the `searcher` to get `k` passages for `questions`:
    ##### YOUR CODE HERE



    # Softmax normalize the scores and convert the list to
    # a NumPy array:
    ##### YOUR CODE HERE



    # Get the passages as a list of texts:
    ##### YOUR CODE HERE




In [ ]:
def test_get_passages_with_scores(func):
    question = "What is linguistics?"        
    passages, passage_probs = get_passages_with_scores(question, k=2)    
    if len(passages) != len(passage_probs):
        print("`get_passages_with_scores` should return equal length "
              "lists of passages and passage probabilities.")
        return
    if len(passages) != 2:
        print(f"`get_passages_with_scores` should return `k` passages. Yours returns {len(passages)}")
        return
    if not all(isinstance(psg, str) for psg in passages):
        print("The first return argument should be a list of passage strings.")
        return
    if not all(isinstance(p, (float, np.float32, np.float64)) for p in passage_probs): 
        print("The second return argument should be a list of floats.")
        return 
    print("No errors detected in `get_passages_with_scores`")

In [ ]:
test_get_passages_with_scores(get_passages_with_scores)

In [ ]:
def answer_scoring(passages, passage_probs, prompts, gen_func=run_eleuther):
    """Implements our basic scoring strategy.

    Parameters
    ----------
    passages : list of str
    passage_probs : list of float
    prompts : list of str
    gen_func : either `run_eleuther` or `run_gpt3`

    Returns
    -------
    list of pairs (score, dict), sorted with the largest score first.
    `dict` should be the return value of `gen_func` for an example.

    """
    data = []
    for passage, passage_prob, prompt in zip(passages, passage_probs, prompts):
        # Run `gen_func` on [prompt] (crucially, the singleton list here),
        # and get the dictionary `gen` from the singleton list `gen_func`
        # returns, and then use the values to score `gen` according to our
        # scoring method.
        #
        # Be sure to use "generated_answer_probs" for the scores.
        ##### YOUR CODE HERE



    # Return `data`, sorted with the highest scoring `(score, gen)`
    # pair given first.
    ##### YOUR CODE HERE




In [ ]:
def test_answer_scoring(func):
    passages = [
        "Pragmatics is the study of language use.", 
        "Phonology is the study of linguistic sound systems."]
    passage_probs = [0.75, 0.25]
    prompts = passages
    
    def gen_func(*prompts):
        return [{
            "generated_answer": "Constant output", 
            "generated_answer_tokens": ["Constant", "output"], 
            "generated_answer_probs": [0.1, 0.2]}]
    
    data = func(passages, passage_probs, prompts, gen_func=gen_func)
    
    if not all(len(x) == 2 for x in data):
        print("`answer_scoring` should return a list of pairs (score, gen)")
        return 
    if not isinstance(data[0][0], (float, np.float32, np.float64)):
        print("The first member of each pair in `data` should be a score (type `float`).")
        return    
    if not isinstance(data[0][1], dict):
        print("The second member of each pair in `data` should be a dict " 
              "created by running `gen_func` on a single example.")
        return    
    if data[0][0] != max([x for x, y in data]):
        print("`answer_scoring` should sort its data with the highest score first.")
        return 
    
    print("No errors detected in `answer_scoring`")

In [ ]:
test_answer_scoring(answer_scoring)

In [ ]:
def answer_scoring_demo(question):
    """Example usage for answer_scoring. Here we extract the top-scoring
    results, which can then be used in an evaluation."""    
    passages, passage_probs = get_passages_with_scores(question)
    prompts = [build_zero_shot_openqa_prompt(question, psg) for psg in passages]
    data = answer_scoring(passages, passage_probs, prompts)
    # Top-scoring answer string:
    return data[0][1]

In [ ]:
answer_scoring_demo("How long is Moby Dick?")

### Your original system [3 points]

This question asks you to design your own few-shot OpenQA system. All of the code above can be used and modified for this, and the requirement is just that you try something new that goes beyond what we've done so far. 

Terms for the bake-off:

* You can make free use of SQuAD and other publicly available data.
* The LM must be an autoregressive language model. No trained QA components can be used. Our list of preallowed models are those available via the OpenAI API whose names begin with "text" and the Eluether models "gpt-neo-125M", "gpt-neo-1.3B", "gpt-neo-2.7B", and "gpt-j-6B". If you would like to use a model outside of this set, please check with the teaching team first.

Here are some ideas for the original system:

* We have so far sampled randomly from the SQuaD train set to create few-shot prompts. One might instead sample passages that have some connection to the target question.

* We have used actual SQuAD training examples to build contexts. These might be different in meaningful ways from the passages in our corpus. An alternative is to use the SQuAD question–answer pairs to retrieve passages that contain the answer and use the resulting question–answer–passage triple when building prompts.

* There are a lot of parameters to our LMs that we have so far ignored. Exploring different values might lead to better results. The `temperature` parameter is highly impactful for our task.

* We have distributed a fixed index of 100K passages. These cover SQuAD plus our bake-off data, but there might still be value in creating a different/expanded index. There is starter code for indexing data with ColBERT [here](https://github.com/stanford-futuredata/ColBERT/blob/new_api/docs/intro.ipynb).

* [Khattab et al. (2021a)](https://aclanthology.org/2021.tacl-1.55/) fine-tune the retriever through a handful of successive rounds, using weak supervision from the QA dataset. This is an ambitious direction that could quickly build to an original project, as the role of retriever training is under-explored so far in the context of few-shot OpenQA.

* In our "Answer scoring" question, we don't normalize scores by answer length. Such normalization might be fairer to long answers and so seems worth adding.

* Our "Answer scoring" question is inspired by the Retrieval Augmented Generation (RAG) model of [Lewis et al. 2020](https://arxiv.org/abs/2005.11401). Their model fully marginalizes over $k$ retrieved passages to create a proper model of $P(\textrm{answer} \mid \textrm{question})$. Implementing this requires having the probabilities for the prompts. For GPT-3, these can be obtained with `echo=False`, which will lead you to have to make changes to the output processing of `run_gpt3`. For the Eleuther models, one needs to do another call to the model forward function. Here is some starter code that could be used to begin modifying `run_eleuther`:

   ```
    prompt_logits = eleuther_model(prompt_ids).logits                
    prompt_probs = prompt_logits.softmax(-1)                                   
    prompt_probs = torch.gather(prompt_probs, 2, prompt_ids[:, :, None]).squeeze(-1)
    prompt_probs = [list(prompt_prob.numpy()) for p in prompt_probs]
   ```

__Original system instructions__:

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. 

We also ask that you report the best macro F1 score your system got during development on `dev_exs` [as defined above](#SQuAD-dev-sample), just to help us understand how systems performed overall.

Please review the descriptions in the following comment and follow the instructions.

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.

## Bake-off [1 point]

For the bake-off, you simply need to be able to run your system on the file 

```data/openqa/cs224u-openqa-test-unlabeled.txt```

The following code should download it for you if necessary:

In [ ]:
if not os.path.exists(os.path.join("data", "openqa", "cs224u-openqa-test-unlabeled.txt")):
    !mkdir -p data/openqa
    !wget https://web.stanford.edu/class/cs224u/data/cs224u-openqa-test-unlabeled.txt -P data/openqa/

If the above fails, you can just download https://web.stanford.edu/class/cs224u/data/cs224u-openqa-test-unlabeled.txt and place it in `data/openqa`.

This file contains only questions. The starter code below will help you structure this. It writes a file "cs224u-openqa-bakeoff-entry.json" to the current directory. That file should be uploaded as-is. Please do not change its name.

In [ ]:
def create_bakeoff_submission():
    filename = os.path.join("data", "openqa", "cs224u-openqa-test-unlabeled.txt")
    
    # This should become a mapping from questions (str) to response
    # dicts from your system.
    gens = {} 
        
    with open(filename) as f:
        questions = f.read().splitlines()
    
    # `questions` is the list of questions you need to evaluate your system on.
    # Put whatever code you need to in here to evaluate your system.
    # All you need to be sure to do is create a list of dicts with at least
    # the keys of the dicts returned by `run_gpt` and `run_eleuther`.
    # Add those dicts to `gens`.
    #
    # Here is an example where we just do "Open QA with no context",
    # for an "original system" that would not earn any credit (since
    # it is not original!):
    for question in questions:
        gens[question] = run_eleuther([question])[0]
        
    # Quick tests we advise you to run: 
    # 1. Make sure `gens` is a dict with the questions as the keys:
    assert all(q in gens for q in questions)
    # 2. Make sure the values are dicts and have the key we will use:
    assert all(isinstance(d, dict) and "generated_answer" in d for d in gens.values())
            
    # And finally the output file:
    with open("cs224u-openqa-bakeoff-entry.json", "wt") as f:
        json.dump(gens, f, indent=4)    

In [ ]:
create_bakeoff_submission()